In [1]:
%%time
from operator import itemgetter
import networkx as nx
import wikipedia 
import matplotlib.pyplot as plt

Topic = "Bubble Tea".title()
degree=2
#ignore non-wiki article pages
IgnoreLinksWith = ("International Standard Serial Number",
    "International Standard Book Number",
    "National Diet Library",
    "International Standard Name Identifier",
    "International Standard Book Number (Identifier)",
    "Pubmed Identifier", 
    "Pubmed Central",
    "Digital Object Identifier", 
    "Arxiv",
    "Proc Natl Acad Sci Usa", 
    "Bibcode",
    "Library Of Congress Control Number", 
    "Jstor",
    "Doi (Identifier)",
    "Isbn (Identifier)",
    "Pmid (Identifier)",
    "Arxiv (Identifier)",
    "Bibcode (Identifier)",
    "Issn (Identifier)",
    "hdl (identifier)")
ToDoList = [(0, Topic)] # Initialize the list of links
ToDoSet = set(Topic) # Make a set of the article title
FinishedSet = set() # Set of parsed article titles
g = nx.DiGraph()
layer, page = ToDoList[0]

Wall time: 1.14 s


In [2]:
%%time

while layer < degree:
# Remove the current page from ToDoList, and add it to FinishedSet 
# it will skip over the page if it sees it again
  
    del ToDoList[0]
    FinishedSet.add(page)
  
  # test intial should be Topic page
    print(layer, page) 
  
  # if page no loady
    try:
        wiki = wikipedia.page(page)
    except:
        layer, page = ToDoList[0]
        print(page,"did not work")
        continue
  
    for link in wiki.links:
        link = link.title()
        #added to also catch other identifier pages
        if link not in IgnoreLinksWith and not link.startswith("List Of") and "(Identifier)" not in link:
            if link not in ToDoSet and link not in FinishedSet:
                ToDoList.append((layer + 1, link))
                ToDoSet.add(link)
            g.add_edge(page, link)
    layer, page = ToDoList[0]

0 Bubble Tea
1 3D Ultrasound
1 Atc Code V08
Abdominal Aortic Aneurysm did not work
1 Abdominal Aortic Aneurysm
1 Abdominal Ultrasonography
1 Abdominal X-Ray
1 Acetrizoic Acid
1 Acoustic Radiation Force
1 Acute Stroke Imaging
1 Ad Hoc
Adipiodone did not work
1 Adipiodone
1 Adis International
1 Air


c:\users\ylk01\appdata\local\programs\python\python39\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\users\ylk01\appdata\local\programs\python\python39\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Alliance Pharmaceutical did not work
1 Alliance Pharmaceutical
1 Angiocardiography
1 Angiogenesis
1 Angiography
Animal Culture did not work
1 Animal Culture
1 Aortography
1 Arthrogram
1 Atherosclerosis
1 Backscatter
1 Barium Sulfate
1 Biological Tissue
1 Biomimicry
1 Biotin
Blood did not work
1 Blood
1 Blood Flow
1 Bone Scintigraphy
1 Bracco (Company)
1 Brain Positron Emission Tomography
1 Breast Mri
1 Breast Ultrasound
1 Bronchography
1 Ct Pulmonary Angiogram
1 Ct Scan
Calcium Iopodate did not work
1 Calcium Iopodate
1 Cancer
1 Carbodiimide
1 Cardiac Pet
1 Cardiac Magnetic Resonance Imaging
1 Cardiac Magnetic Resonance Imaging Perfusion
1 Carotid Ultrasonography
1 Cell Adhesion Molecules
1 Chest Radiograph
1 Cholangiography
1 Cholecystography
1 Cholescintigraphy
1 Circulatory System
Clinical Trial did not work
1 Clinical Trial
1 Compressibility
Computed Tomography Angiography did not work
1 Computed Tomography Angiography
1 Computed Tomography Of The Abdomen And Pelvis
1 Computed Tomo

In [3]:
%%time
# remove self loops
g.remove_edges_from(nx.selfloop_edges(g))

# identify duplicates like "apple" and "apples"
duplicates = [(node, node + "s")
              for node in g if node + "s" in g
             ]

for dup in duplicates:
  # *dup is a technique named "unpacking"
  g = nx.contracted_nodes(g, *dup, self_loops=False)

print(duplicates)

#more filterings: page "x-y" is the same as "x y" 
duplicates = [(x, y) for x, y in 
              [(node, node.replace("-", " ")) for node in g]
                if x != y and y in g]
print(duplicates)

for dup in duplicates:
    g = nx.contracted_nodes(g, *dup, self_loops=False)

    # nx.contracted creates a new node/edge attribute called contraction
    # the value of the attribute is a dictionary, but GraphML
    # does not support dictionary attributes
nx.set_node_attributes(g, 0,"contraction")
nx.set_edge_attributes(g, 0,"contraction")

[('Biological Tissue', 'Biological Tissues'), ('Cancer', 'Cancers'), ('Clinical Trial', 'Clinical Trials'), ('Endothelial Cell', 'Endothelial Cells'), ('Fatty Acid', 'Fatty Acids'), ('Gamma Ray', 'Gamma Rays'), ('Immunoglobulin', 'Immunoglobulins'), ('Integrin', 'Integrins'), ('Laser', 'Lasers'), ('Lipid', 'Lipids'), ('Mri Sequence', 'Mri Sequences'), ('Microparticle', 'Microparticles'), ('Microsphere', 'Microspheres'), ('Nanoparticle', 'Nanoparticles'), ('Perfluorocarbon', 'Perfluorocarbons'), ('Perfluorocarbon Emulsion', 'Perfluorocarbon Emulsions'), ('Phospholipid', 'Phospholipids'), ('Red Blood Cell', 'Red Blood Cells'), ('Selectin', 'Selectins'), ('X-Ray', 'X-Rays'), ('Apgar Score', 'Apgar Scores'), ('Aneurysm', 'Aneurysms'), ('Aortic Aneurysm', 'Aortic Aneurysms'), ('Arteriole', 'Arterioles'), ('Arteriovenous Malformation', 'Arteriovenous Malformations'), ('Cardiovascular Disease', 'Cardiovascular Diseases'), ('Foam Cell', 'Foam Cells'), ('Hemorrhoid', 'Hemorrhoids'), ('Infection

In [6]:
%%time
# filter nodes by degree
core = [node for node, deg in dict(g.degree()).items() if deg >= degree]

# select a subgraph with filtered nodes
gsub = nx.subgraph(g, core)

print("{} nodes, {} edges".format(len(gsub), nx.number_of_edges(gsub)))

# convert graph to gelphi format
nx.write_gexf(gsub, "wikipedia.gexf")

7931 nodes, 51393 edges
Wall time: 1.79 s
